# Extract Fine-tuned Model Weights

In [1]:
# Load fine-tuned model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gokuls/BERT-tiny-emotion-intent")
model = AutoModelForSequenceClassification.from_pretrained("gokuls/BERT-tiny-emotion-intent")

model.eval()

/Users/tonyma/code/FHE-BERT-Tiny-Emotion/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [2]:
import os
import numpy as np

# Create a directory to save the weights
os.makedirs('../weights-emotion', exist_ok=True)

# QKV & Pooler Weights
def save_weight_flat_R(weight, name):
    weight_np = weight.cpu().detach().numpy()
    
    flattened = weight_np.T.flatten()
    
    num_full_rows = len(flattened) // 128
    
    reshaped_weight = flattened[:num_full_rows * 128].reshape(-1, 128)
    
    np.savetxt(f"../weights-emotion/{name}.txt", reshaped_weight, delimiter=',')

# Selfoutput Weights
def save_weight_flat_C(weight, name):
    weight_np = weight.cpu().detach().numpy()
    
    flattened = weight_np.flatten()
    
    num_full_rows = len(flattened) // 128
    
    reshaped_weight = flattened[:num_full_rows * 128].reshape(-1, 128)
    
    np.savetxt(f"../weights-emotion/{name}.txt", reshaped_weight, delimiter=',')
    
# Save Intermediate/Output weights
def save_weight_flat_split(weight, name):
    weight_np = weight.cpu().detach().numpy()
    
    # If intermediate weights
    if (weight_np.shape == (512, 128)):
        # Transpose to (128, 512)
        weight_np = weight_np.T
    
    # Split into 4 parts of 128x128
    for i in range(4):
        part = weight_np[:, i*128:(i+1)*128]
        np.savetxt(f"../weights-emotion/{name}{i+1}.txt", part.flatten(), delimiter=',')

# Save Xlassifier weights and each layer Biases directly
def save_layer_direct(layer, name):
    layer_np = layer.cpu().detach().numpy()
    
    # Save the entire weight
    np.savetxt(f"../weights-emotion/{name}.txt", layer_np, delimiter=',')

# Save QKV weights
save_weight_flat_R(model.bert.encoder.layer[0].attention.self.query.weight, "layer0_attself_query_weight")
save_weight_flat_R(model.bert.encoder.layer[0].attention.self.key.weight, "layer0_attself_key_weight")
save_weight_flat_R(model.bert.encoder.layer[0].attention.self.value.weight, "layer0_attself_value_weight")
save_weight_flat_R(model.bert.encoder.layer[1].attention.self.query.weight, "layer1_attself_query_weight")
save_weight_flat_R(model.bert.encoder.layer[1].attention.self.key.weight, "layer1_attself_key_weight")
save_weight_flat_R(model.bert.encoder.layer[1].attention.self.value.weight, "layer1_attself_value_weight")
# Save Selfoutput weights
save_weight_flat_C(model.bert.encoder.layer[0].attention.output.dense.weight, "layer0_selfoutput_weight")
save_weight_flat_C(model.bert.encoder.layer[1].attention.output.dense.weight, "layer1_selfoutput_weight")
# Save Intermediate weights
save_weight_flat_split(model.bert.encoder.layer[0].intermediate.dense.weight, "layer0_intermediate_weight")
save_weight_flat_split(model.bert.encoder.layer[1].intermediate.dense.weight, "layer1_intermediate_weight")
# Save Output weights
save_weight_flat_split(model.bert.encoder.layer[0].output.dense.weight, "layer0_output_weight")
save_weight_flat_split(model.bert.encoder.layer[1].output.dense.weight, "layer1_output_weight")
# Save Pooler weights
save_weight_flat_R(model.bert.pooler.dense.weight, "pooler_dense_weight")
# Save Classifier weights
save_layer_direct(model.classifier.weight, "classifier_weight")

# Save QKV biases
save_layer_direct(model.bert.encoder.layer[0].attention.self.query.bias, "layer0_attself_query_bias")
save_layer_direct(model.bert.encoder.layer[0].attention.self.key.bias, "layer0_attself_key_bias")
save_layer_direct(model.bert.encoder.layer[0].attention.self.value.bias, "layer0_attself_value_bias")
save_layer_direct(model.bert.encoder.layer[1].attention.self.query.bias, "layer1_attself_query_bias")
save_layer_direct(model.bert.encoder.layer[1].attention.self.key.bias, "layer1_attself_key_bias")
save_layer_direct(model.bert.encoder.layer[1].attention.self.value.bias, "layer1_attself_value_bias")
# Save Selfoutput biases
save_layer_direct(model.bert.encoder.layer[0].attention.output.dense.bias, "layer0_selfoutput_bias")
save_layer_direct(model.bert.encoder.layer[1].attention.output.dense.bias, "layer1_selfoutput_bias")
# Save intermediate biases
save_layer_direct(model.bert.encoder.layer[0].intermediate.dense.bias, "layer0_intermediate_bias")
save_layer_direct(model.bert.encoder.layer[1].intermediate.dense.bias, "layer1_intermediate_bias")
# Save Output biases
save_layer_direct(model.bert.encoder.layer[0].output.dense.bias, "layer0_output_bias")
save_layer_direct(model.bert.encoder.layer[1].output.dense.bias, "layer1_output_bias")
# Save Pooler biases
save_layer_direct(model.bert.pooler.dense.bias, "pooler_dense_bias")
# Save Classifier biases
save_layer_direct(model.classifier.bias, "classifier_bias")

print("Weights and Biases saved successfully.")

Weights and Biases saved successfully.
